## Código para identificar tiendas Cornershop por código postal

Con base en los códigos postales de SEPOMEX, se identifican las tiendas Cornershop que se encuentran en cada uno de los códigos postales.

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time
import requests
import random

In [2]:
#Directorio de trabajo
try:
    os.chdir('C:/Users/claudio.pacheco/Documents/GitHub/pacic/')
except:
    os.chdir("D:/")

#### Descarga de datos de Sepomex

In [3]:
#Descarga de códigos postales de Ciudad de México
url = 'https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx'
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
combo = driver.find_element(By.CLASS_NAME, "cbo")
select = Select(combo)
select.select_by_visible_text("Ciudad de México")
#Descarga en carpeta de trabajo
driver.find_element(By.ID, "btnDescarga").click()
time.sleep(5)
driver.close()


C:\Users\claudio.pacheco\AppData\Local\Temp\ipykernel_16344\1557182785.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver.exe")


In [6]:

#Mover a carpeta de trabajo el archivo descargado

#os.rename('C:/Users/claudio.pacheco/Downloads/Ciudad de México.xls', 'C:/Users/claudio.pacheco/Documents/GitHub/pacic/cp/Ciudad de México.xls')

In [6]:
#Leer archivo descargado
cp = pd.read_excel('cp/Ciudad de México.xls',sheet_name="Distrito_Federal",usecols=["d_codigo", "d_asenta", 
                                                                                    "d_tipo_asenta", "D_mnpio", 
                                                                                    "c_mnpio","d_zona"],dtype={"d_codigo":str})

In [7]:
cp

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,01010,Los Alpes,Colonia,Álvaro Obregón,10,Urbano
2,01020,Guadalupe Inn,Colonia,Álvaro Obregón,10,Urbano
3,01030,Axotla,Pueblo,Álvaro Obregón,10,Urbano
4,01030,Florida,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...
1517,16840,Santa Cruz Chavarrieta,Colonia,Xochimilco,13,Urbano
1518,16850,Chapultepec,Barrio,Xochimilco,13,Urbano
1519,16860,Santa Cruz de Guadalupe,Colonia,Xochimilco,13,Urbano
1520,16880,Santa Cecilia Tepetlapa,Pueblo,Xochimilco,13,Urbano


In [8]:
#Eliminar duplicados
cp = cp.drop_duplicates(subset=['d_codigo'], keep='first')
cp

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,01010,Los Alpes,Colonia,Álvaro Obregón,10,Urbano
2,01020,Guadalupe Inn,Colonia,Álvaro Obregón,10,Urbano
3,01030,Axotla,Pueblo,Álvaro Obregón,10,Urbano
5,01040,Campestre,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...
1517,16840,Santa Cruz Chavarrieta,Colonia,Xochimilco,13,Urbano
1518,16850,Chapultepec,Barrio,Xochimilco,13,Urbano
1519,16860,Santa Cruz de Guadalupe,Colonia,Xochimilco,13,Urbano
1520,16880,Santa Cecilia Tepetlapa,Pueblo,Xochimilco,13,Urbano


### Búsqueda de tiendas Cornershop por código postal

In [9]:
#Función para obtener tiendas por código postal
def zip_code(zip_code:str):
    url="https://cornershopapp.com/api/v1/collections/13?locality="+zip_code+"&country=MX"
    r = requests.get(url)
    #Si no se encuentra, lanza una df en blanco
    if r.status_code == 404:
        df=pd.DataFrame({"id_tienda":[],"tienda":[],"codigo_postal":[]})
        return df
    #Caso contrario, lanza un df con las tiendas
    else:
        data_zip = r.json()
        tiendas=[]
        ids=[]
        codigo=zip_code
        for stores in data_zip["branches"]:
            tiendas.append(stores["name"])
            ids.append(stores["id"])
    #Llevarlo a un dataframe
    df=pd.DataFrame({"id_tienda":ids,"tienda":tiendas,"codigo_postal":codigo})
    return df

In [10]:
tiendascorner=pd.DataFrame()
for codigos in range(len(cp["d_codigo"])): 
    time.sleep(random.randint(1,2))
    codigo = cp["d_codigo"].get(codigos)
    if codigo is not None:
        tiendascorner = pd.concat([tiendascorner, zip_code(codigo)], axis=0)


In [11]:
tiendascorner

,id_tienda,tienda,codigo_postal
0,1445,Chedraui,01000
1,49,City Market,01000
2,92,Costco,01000
3,4414,La Comer,01000
4,10002,Soriana,01000
...,...,...,...
2,92,Costco,11950
3,4412,La Comer,11950
4,10362,Soriana,11950
5,4432,Fresko,11950


In [12]:
#pegar con datos de cp
tiendascorner = pd.merge(tiendascorner, cp, left_on='codigo_postal', right_on='d_codigo', how='left')

In [13]:
tiendascorner

,id_tienda,tienda,codigo_postal,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,1445,Chedraui,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,49,City Market,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
2,92,Costco,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
3,4414,La Comer,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
4,10002,Soriana,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...,...,...,...
5474,92,Costco,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5475,4412,La Comer,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5476,10362,Soriana,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5477,4432,Fresko,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano


In [27]:
#Exportar a csv
tiendascorner.to_csv("tiendascorner.csv",index=False)

In [36]:
#Códigos únicos para búsqueda de tiendas
busqueda=tiendascorner.copy()
busqueda=busqueda.drop_duplicates(subset=['id_tienda'], keep='first')

In [38]:
#Completar registros con información de cornershop
url_basica="https://cornershopapp.com/api/v3/branches/"
url_fin="?with_suspended_slots&app_capabilities=DATA_REQUEST_BASIC_FORM,CUSTOMER_WITH_SENDBIRD_CHAT,DATA_REQUEST_PRESCRIPTION,LOYALTY_PROVIDERS_WITH_2FA,IMAGE_SUPPORT_CHAT,TC_BLOCKER,CUSTOMER_SMART_SERVICE_FEE,STORE_WITHOUT_PRICING_NOTES"
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

In [48]:
id_tienda=[]
sucursales=[]
latitud=[]
longitud=[]
store_name=[]
store_id=[]
address=[]


#buscar los id_tiendas de tiendascorner en la api de cornershop
for tiendas in busqueda["id_tienda"]:
    url = url_basica+str(tiendas)+url_fin
    r = requests.get(url, headers=headers)
    #Si el código es 404 o 429, se salta el registro
    if r.status_code == 404 or r.status_code == 429:
        continue
    #Si el código es 200, se extrae la información
    else:
        data_branches = r.json()
        id_tienda.append(data_branches['branch']['id'])
        sucursales.append(data_branches['branch']['name'])
        latitud.append(data_branches['branch']['lat'])
        longitud.append(data_branches['branch']['lng'])
        store_name.append(data_branches['branch']['store_name'])
        store_id.append(data_branches['branch']['store_id'])
        address.append(data_branches['branch']['address'])
        time.sleep(random.randint(1,2))
        

In [60]:
#pasa los datos a un dataframe
df=pd.DataFrame({"id_tienda":id_tienda,"sucursal":sucursales,"latitud":latitud,"longitud":longitud,"store_name":store_name,"store_id":store_id,"address":address})

In [61]:
df

,id_tienda,sucursal,latitud,longitud,store_name,store_id,address
0,1445,Pedregal,19.335124,-99.198477,Selecto Chedraui,22,Av. San Jerónimo 236
1,49,San Jerónimo,19.335648,-99.195824,City Market,9,Av San Jerónimo 190
2,92,Mixcoac,19.387269,-99.190934,Costco,7,Blvd. Adolfo Lopez Mateos 1181
3,4414,La Comer Coyoacán,19.345009,-99.171911,La Comer,1423,"MIGUEL ANGEL DE QUEVEDO 443, ROMERO DE TERRERO..."
4,10002,San Jeronimo,19.330351,-99.208839,Soriana,3729,"Anillo Perif. 630, Jardines del Pedregal, Álva..."
...,...,...,...,...,...,...,...
70,20638,Chedraui Selecto Tezontle,19.384580,-99.082231,Chedraui,22,"Av Canal de Tezontle 1512, Área Federal Centra..."
71,27464,La Comer Iztapalapa Las Torres,19.352049,-99.013243,La Comer,1423,"CALZ ERMITA IZTAPALAPA 3865, SANTA MARIA AZTAH..."
72,26729,Chedraui Neza Estadio,19.400626,-98.987260,Chedraui,22,4Ta. Avenida No. 257 Lote 1 Entre Lázaro Cárde...
73,26722,Chedraui Tepozán,19.362166,-98.988547,Chedraui,22,Av. Tepozanes No. 3 Entre Tabachines Y Siervo ...


In [62]:
#Pegar con tiendascorner
tiendascorner_final = pd.merge(tiendascorner, df, left_on='id_tienda', right_on='id_tienda', how='left')

In [64]:
print(len(tiendascorner_final))
tiendascorner_final

5479


,id_tienda,tienda,codigo_postal,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona,sucursal,latitud,longitud,store_name,store_id,address
0,1445,Chedraui,1000,1000,San Ángel,Colonia,Álvaro Obregón,10,Urbano,Pedregal,19.335124,-99.198477,Selecto Chedraui,22,Av. San Jerónimo 236
1,49,City Market,1000,1000,San Ángel,Colonia,Álvaro Obregón,10,Urbano,San Jerónimo,19.335648,-99.195824,City Market,9,Av San Jerónimo 190
2,92,Costco,1000,1000,San Ángel,Colonia,Álvaro Obregón,10,Urbano,Mixcoac,19.387269,-99.190934,Costco,7,Blvd. Adolfo Lopez Mateos 1181
3,4414,La Comer,1000,1000,San Ángel,Colonia,Álvaro Obregón,10,Urbano,La Comer Coyoacán,19.345009,-99.171911,La Comer,1423,"MIGUEL ANGEL DE QUEVEDO 443, ROMERO DE TERRERO..."
4,10002,Soriana,1000,1000,San Ángel,Colonia,Álvaro Obregón,10,Urbano,San Jeronimo,19.330351,-99.208839,Soriana,3729,"Anillo Perif. 630, Jardines del Pedregal, Álva..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5474,92,Costco,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano,Mixcoac,19.387269,-99.190934,Costco,7,Blvd. Adolfo Lopez Mateos 1181
5475,4412,La Comer,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano,La Comer Lomas Anahuac,19.399494,-99.276637,La Comer,1423,"AV MAGNOCENTRO LT 1 MZ 2 , SAN FERNANDO HUIXQU..."
5476,10362,Soriana,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano,Tacubaya,19.408929,-99.184396,Soriana,3729,Gral Jose Moran 3
5477,4432,Fresko,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano,Fresko Tecamachalco,19.421548,-99.238215,Fresko,45,"AV DE LAS FUENTES 190, LOMAS DE TECAMACHALCO, ..."


In [65]:
#Guardar en csv
tiendascorner_final.to_csv("tiendascorner_final.csv",index=False,encoding="latin1")